In [ ]:
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import os
from matplotlib import pyplot as plt
import sqlalchemy as sa
import seaborn as sns
import datetime
color_palette = sns.color_palette()

In [ ]:
load_dotenv()
engine = sa.create_engine(os.getenv("EL_NINO_DB_URL"))
output_figures_path = os.getenv("FIGURE_OUTPUT")

In [ ]:
raw_precipitation_df = pd.read_sql('SELECT * FROM raw_precipitation;', engine)

In [ ]:
raw_precipitation_df.set_index('event_calendar_date', inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
raw_precipitation_df[raw_precipitation_df['source_id'] == 'SenamhiLaEsperanzaStation'].plot(ax=ax, c=color_palette[0])
ax.legend(['Senamhi LaEsperanza Station'])
leg = ax.get_legend()
leg.legend_handles[0].set_color(color_palette[0])
ax.set_ylim([-100, 200])
ax.set_xlim([datetime.date(1960,1,1), datetime.date(2020,12,31)])
plt.show()
fig.savefig(f'{output_figures_path}/f04_01_original_data.png')   # save the figure to file



In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
raw_precipitation_df[raw_precipitation_df['source_id'] == 'ServicioNacionalMeteorologiaHidrologa'].plot(ax=ax, c=color_palette[2])
raw_precipitation_df[raw_precipitation_df['source_id'] == 'SenamhiMallaresStation'].plot(ax=ax, c=color_palette[1])
raw_precipitation_df[raw_precipitation_df['source_id'] == 'SenamhiPanangaStation'].plot(ax=ax, c=color_palette[4])
raw_precipitation_df[raw_precipitation_df['source_id'] == 'SenamhiLaEsperanzaStation'].plot(ax=ax, c=color_palette[0])
ax.legend(['Servicio Nacional de Meteorologia e Hidrologia',
           'Senamhi Mallares Station',
           'Senamhi Pananga Station',
           'Senamhi LaEsperanza Station'])
leg = ax.get_legend()
leg.legend_handles[0].set_color(color_palette[2])
leg.legend_handles[1].set_color(color_palette[1])
leg.legend_handles[2].set_color(color_palette[4])
leg.legend_handles[3].set_color(color_palette[0])
ax.set_ylim([-100, 200])
ax.set_xlim([datetime.date(1960,1,1), datetime.date(2020,12,31)])
plt.show()
fig.savefig(f'{output_figures_path}/f04_02_multiple_data.png')   # save the figure to file



In [ ]:
raw_precipitation_df['precipitation_number_nan'] = np.where(raw_precipitation_df['precipitation_number'] < 0.0, np.nan, raw_precipitation_df['precipitation_number'])
target_df = raw_precipitation_df[raw_precipitation_df['source_id'] == 'SenamhiLaEsperanzaStation'][['precipitation_number_nan']].copy()
target_df.rename(columns = {"precipitation_number_nan": "precipitation_number_target"}, inplace=True)
hourly_df = raw_precipitation_df[raw_precipitation_df['source_id'] == 'ServicioNacionalMeteorologiaHidrologa'][['precipitation_number_nan']].copy()
hourly_df.rename(columns = {"precipitation_number_nan": "precipitation_number_hourly"}, inplace=True)
station2_df = raw_precipitation_df[raw_precipitation_df['source_id'] == 'SenamhiMallaresStation'][['precipitation_number_nan']].copy()
station2_df.rename(columns = {"precipitation_number_nan": "precipitation_number_station_2"}, inplace=True)
station3_df = raw_precipitation_df[raw_precipitation_df['source_id'] == 'SenamhiPanangaStation'][['precipitation_number_nan']].copy()
station3_df.rename(columns = {"precipitation_number_nan": "precipitation_number_station_3"}, inplace=True)

result_df = target_df.merge(hourly_df, left_index=True, right_index=True, how='outer'). \
    merge(station2_df, left_index=True, right_index=True, how='left'). \
    merge(station3_df, left_index=True, right_index=True, how='left')


In [ ]:
result_df['fixed_target_precipitation'] = np.where(result_df['precipitation_number_target'].notnull(), result_df['precipitation_number_target'],
                                    np.where(result_df['precipitation_number_hourly'].notnull(), result_df['precipitation_number_hourly'],
                                    np.where(result_df['precipitation_number_station_2'].isnull(), result_df['precipitation_number_station_3'],
                                    np.where(result_df['precipitation_number_station_3'].isnull(), result_df['precipitation_number_station_2'],
                                             np.maximum(result_df['precipitation_number_station_2'], result_df['precipitation_number_station_3'])))))

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
result_df[['fixed_target_precipitation']].plot(ax=ax, color=color_palette[0])
ax.legend(['Target Precipitation'])
leg = ax.get_legend()
leg.legend_handles[0].set_color(color_palette[0])
ax.set_ylim([-100, 200])
ax.set_xlim([datetime.date(1960,1,1), datetime.date(2020,12,31)])
plt.show()
fig.savefig(f'{output_figures_path}/f04_03_target_precipitation.png')



In [ ]:
master_precipitation_df = result_df.rename(
    columns={"fixed_target_precipitation": "precipitation_number"})[['precipitation_number']]

In [ ]:
master_precipitation_df.head()

In [ ]:
master_precipitation_df.to_sql('master_precipitation', engine, if_exists='append')

In [ ]:
read_master_precipitation_df = pd.read_sql('SELECT * FROM master_precipitation;', engine)
read_master_precipitation_df.set_index('event_calendar_date', inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
read_master_precipitation_df.plot(ax=ax, color=color_palette[0])
ax.legend(['Read Target Precipitation'])
leg = ax.get_legend()
leg.legend_handles[0].set_color(color_palette[0])
ax.set_ylim([-100, 200])
ax.set_xlim([datetime.date(1960,1,1), datetime.date(2020,12,31)])
plt.show()


